In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss
import logging
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.embeddings import fetch_GloVe
from web.evaluate import evaluate_similarity
from web.embedding import Embedding, Vocabulary
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from python_speech_features.base import mfcc

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# https://s3.amazonaws.com/fast-ai-nlp/wikitext-103.tgz

In [3]:
df_train = pd.read_csv('data/wikitext-103/train.csv', header=None)

In [4]:
df_train.shape

(29539, 1)

In [5]:
texts = df_train[0].tolist()
texts = texts[:2000]

In [6]:
from spacy.tokenizer import Tokenizer

In [7]:
import spacy
from spacy.attrs import ORTH, LEMMA

nlp = spacy.load("en_core_web_sm")

nlp.tokenizer.add_special_case('<unk>', [{ORTH: "<unk>"}])
nlp.tokenizer.add_special_case('<eos>', [{ORTH: "<eos>"}])

In [8]:
from joblib import Parallel, delayed
from functools import partial
from spacy.util import minibatch
import spacy

nlp = spacy.load("en_core_web_sm")

batch_size= 200
partitions = minibatch(texts, size=batch_size)

executor = Parallel(n_jobs=10, backend="multiprocessing", prefer="threads")

def transform_texts(nlp, texts):
    for i in range(len(texts)):
        texts[i] = [line.strip() for line in texts[i].split('\n') if len(line) > 100]
        texts[i] = ('\n').join(texts[i])
    docs = nlp.tokenizer.pipe(texts)
    return list(docs)

In [9]:
%%time

do = delayed(partial(transform_texts, nlp))
tasks = (do(batch) for batch in partitions)
docs_hierarchical = executor(tasks)

docs = []
for ds in docs_hierarchical: docs += ds

CPU times: user 1min 4s, sys: 3.7 s, total: 1min 8s
Wall time: 1min 11s


In [10]:
%%time
words = []

for doc in docs:
    for token in doc:
        if (token.is_alpha or token.is_punct) and '@' not in str(token):
            words.append(str(token).lower())

CPU times: user 17.1 s, sys: 268 ms, total: 17.4 s
Wall time: 17.4 s


In [11]:
from collections import Counter

In [12]:
c = Counter(words)

In [13]:
tasks = {
    "MEN": fetch_MEN(),
    "WS353": fetch_WS353(),
    "SIMLEX999": fetch_SimLex999()
}

In [14]:
words = []

for k, v in tasks.items():
    for row in v.X:
        for w in row:
            words.append(w)

In [15]:
len(set(words))

1819

In [16]:
len(set([w for w, _ in c.most_common(20_000)]).intersection(words))

1584

In [17]:
vocab = set([w for w, _ in c.most_common(20_000)] + ['<eos>', '<unk>'])

In [18]:
# mkdir -p data/audio_wikitext103_20k

In [19]:
# from time import sleep

In [20]:
# %%time

# from google.cloud import texttospeech

# client = texttospeech.TextToSpeechClient()

# for i, word in enumerate(vocab):
#     if i % 300 == 0: sleep(60)

#     synthesis_input = texttospeech.SynthesisInput(text=word)

#     voice = texttospeech.VoiceSelectionParams(
#         language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
#     )

#     audio_config = texttospeech.AudioConfig(
#         audio_encoding=texttospeech.AudioEncoding.MP3
#     )

#     response = client.synthesize_speech(
#         input=synthesis_input, voice=voice, audio_config=audio_config
#     )

#     # The response's audio_content is binary.
#     with open(f"data/audio_wikitext103_20k/{word}.mp3", "wb") as out:
#         out.write(response.audio_content)

In [21]:
# import librosa

In [22]:
# %%time

# import warnings
# warnings.filterwarnings('ignore')

# word2ary = {}

# for word in vocab:
#     ary, _ = librosa.load(f'data/audio_wikitext103_20k/{word}.mp3', sr=16000)
#     word2ary[word] = ary
    
# pd.to_pickle(word2ary, 'word2ary.pkl')

In [23]:
word2ary = pd.read_pickle('word2ary.pkl')

In [24]:
word2features = {}
for k, v in word2ary.items():
    word2features[k] = mfcc(v)

In [25]:
len(word2features)

20002

In [26]:
lengths = []
for k, v in word2features.items():
    lengths.append(v.shape[0])

In [27]:
max(lengths)

193

In [28]:
def prepare_features(word, pad_to=max(lengths)):
    ary = word2features[word][:pad_to]
    example = np.zeros((pad_to, 13))
    example[:ary.shape[0], :] = ary
    return example.astype(np.float32)

In [29]:
dataset_mean = -2
dataset_std = 10

def normalize_data(ary):
    return (ary - dataset_mean) / dataset_std

In [30]:
word2features_norm = {}
for word in vocab:
    word2features_norm[word] = normalize_data(prepare_features(word))

In [31]:
text = ''

for doc in docs:
    toks = []
    for token in doc:
        if (token.is_alpha or token.is_punct) and '@' not in str(token):
            if str(token) in vocab:
                toks.append(str(token).lower())
            else:
                toks.append('<unk>')
    toks += ['<eos>']
    text += ' '.join(toks)
    text += ' '

In [32]:
len(text)

32498344

In [33]:
toks = text.split()
len(toks)

6224583

In [34]:
toks[:10]

['<unk>', 'is', 'a', 'fungal', 'genus', 'in', 'the', 'family', '<unk>', ',']

In [35]:
vocab = list(vocab)
# stoi = {s: i for i, s in enumerate(vocab)}

In [36]:
# toks_numeric = [stoi[tok] for tok in toks]
# toks_numeric[:10]

In [37]:
import math
from torch.utils.data import Dataset

class WordDataset(Dataset):

    def __init__(self, data, block_size):
        data_size, vocab_size = len(data), len(vocab)
        print('data has %d words, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { word:i for i,word in enumerate(vocab) }
        self.itos = { i:word for i,word in enumerate(vocab) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size
 
    def __getitem__(self, idx):
        chunk = self.data[idx:idx + self.block_size + 1]
        features = [word2features_norm[s] for s in chunk[:-1]]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return np.stack(features), y

In [50]:
model.encoder

LSTM(13, 256, batch_first=True, bidirectional=True)

In [38]:
block_size = 128

train_dataset = WordDataset(toks[:6_000_000], block_size)
valid_dataset = WordDataset(toks[6_000_000:], block_size)

data has 6000000 words, 20002 unique.
data has 224583 words, 20002 unique.


In [39]:
import math
import logging

import torch
import torch.nn as nn
from torch.nn import functional as F

logger = logging.getLogger(__name__)

class GPTConfig:
    """ base GPT config, params common to all GPT versions """
    embd_pdrop = 0.1
    resid_pdrop = 0.1
    attn_pdrop = 0.1

    def __init__(self, vocab_size, block_size, **kwargs):
        self.vocab_size = vocab_size
        self.block_size = block_size
        for k,v in kwargs.items():
            setattr(self, k, v)

class GPT1Config(GPTConfig):
    """ GPT-1 like network roughly 125M params """
    n_layer = 12
    n_head = 12
    n_embd = 768

class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_drop = nn.Dropout(config.attn_pdrop)
        self.resid_drop = nn.Dropout(config.resid_pdrop)
        # output projection
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head

    def forward(self, x, layer_past=None):
        B, T, C = x.size()

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_drop(self.proj(y))
        return y

class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class GPT(nn.Module):
    """  the full GPT language model, with a context size of block_size """

    def __init__(self, config):
        super().__init__()
        
        self.encoder= nn.LSTM(
            input_size=13,
            hidden_size=config.n_embd // 2,
            num_layers=1,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )
        
        # input embedding stem
#         self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)
        # transformer
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        # decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)

        logger.info("number of parameters: %e", sum(p.numel() for p in self.parameters()))

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)
                elif pn.startswith('encoder'):
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, features, targets=None):        
        b, t, _, _ = features.shape
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."
        
        t_embs = []
        for i in range(b):
            _, (embeddings, _) = self.encoder(features[i])        
            t_embs.append(torch.cat((embeddings[-1], embeddings[-2]), 1))

        token_embeddings = torch.stack(t_embs)
        
        # forward the GPT model
#         token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
        position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
        x = self.drop(token_embeddings + position_embeddings)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))

        return logits, loss

In [40]:
  
"""
Simple training loop; Boilerplate that could apply to any arbitrary neural network,
so nothing in this file really has anything to do with GPT specifically.
"""

import math
import logging

from tqdm import tqdm
import numpy as np

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.dataloader import DataLoader

logger = logging.getLogger(__name__)

class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        logger.info("saving %s", self.config.ckpt_path)
        torch.save(raw_model.state_dict(), self.config.ckpt_path)

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:
                
                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())

                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")

            if not is_train:
                test_loss = float(np.mean(losses))
                logger.info("test loss: %f", test_loss)
                return test_loss

        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay
        for epoch in range(config.max_epochs):

            run_epoch('train')
            if self.test_dataset is not None:
                test_loss = run_epoch('test')

            # supports early stopping based on the test loss, or just save always if no test set is provided
            good_model = self.test_dataset is None or test_loss < best_loss
            if self.config.ckpt_path is not None and good_model:
                best_loss = test_loss
                self.save_checkpoint()

In [41]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

In [42]:
tconf = TrainerConfig(max_epochs=1, batch_size=32, learning_rate=3e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=8)
trainer = Trainer(model, train_dataset, valid_dataset, tconf)
trainer.train()

epoch 1 iter 187495: train loss 2.30185. lr 1.500016e-04: 100%|██████████| 187496/187496 [67:17:22<00:00,  1.29s/it]   


In [43]:
torch.save(model.state_dict(), 'models/gpt_audio.pkl')

In [44]:
# def top_k_logits(logits, k):
#     v, ix = torch.topk(logits, k)
#     out = logits.clone()
#     out[out < v[:, [-1]]] = -float('Inf')
#     return out

# @torch.no_grad()
# def sample(model, x, steps, temperature=1.0, sample=False, top_k=None):
#     """
#     take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
#     the sequence, feeding the predictions back into the model each time. Clearly the sampling
#     has quadratic complexity unlike an RNN that is only linear, and has a finite context window
#     of block_size, unlike an RNN that has an infinite context window.
#     """
#     block_size = model.get_block_size()
#     model.eval()
#     for k in range(steps):
#         x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
#         logits, _ = model(x_cond)
#         # pluck the logits at the final step and scale by temperature
#         logits = logits[:, -1, :] / temperature
#         # optionally crop probabilities to only the top k options
#         if top_k is not None:
#             logits = top_k_logits(logits, top_k)
#         # apply softmax to convert to probabilities
#         probs = F.softmax(logits, dim=-1)
#         # sample from the distribution or take the most likely
#         if sample:
#             ix = torch.multinomial(probs, num_samples=1)
#         else:
#             _, ix = torch.topk(probs, k=1, dim=-1)
#         # append to the sequence and continue
#         x = torch.cat((x, ix), dim=1)

#     return x

In [42]:
# context = ["the", "ship"]
# x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)

In [45]:
# y = sample(model, x, 400, temperature=1.0, sample=True, top_k=10)[0]
# completion = ' '.join([train_dataset.itos[int(i)] for i in y])
# print(completion)

## Evaluate embeddings

In [80]:
import logging
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.embeddings import fetch_GloVe
from web.evaluate import evaluate_similarity
from web.embedding import Embedding, Vocabulary
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [81]:
tasks = {
    "MEN": fetch_MEN(),
    "WS353": fetch_WS353(),
    "SIMLEX999": fetch_SimLex999()
}

In [82]:
speech2vec = KeyedVectors.load_word2vec_format('../speech2vec-pretrained-vectors/speech2vec/50.vec', binary=False) 

In [83]:
class EmbeddingsDataset(Dataset):
    def __len__(self):
        return len(vocab)
    def __getitem__(self, idx):
        return word2features_norm[vocab[idx]]

In [84]:
pred_dl = DataLoader(EmbeddingsDataset(), 128)

In [85]:
%%time

vocab_embeddings = []

model.train(False);
with torch.no_grad():
    for batch in pred_dl:
        _, (embeddings, _) = model.encoder(batch.cuda())
        embeddings = torch.cat((embeddings[-1], embeddings[-2]), 1).cpu().detach()
        vocab_embeddings.append(embeddings)
        
vocab_embeddings = torch.cat(vocab_embeddings)

CPU times: user 1.52 s, sys: 128 ms, total: 1.64 s
Wall time: 1.57 s


In [86]:
our_embeddings = Embedding(
    Vocabulary(vocab),
    vocab_embeddings
)

speech2vec = KeyedVectors.load_word2vec_format('../speech2vec-pretrained-vectors/word2vec/50.vec', binary=False) 
speech2vec_embeddings = Embedding(Vocabulary(list(speech2vec.vocab.keys())), speech2vec.vectors)

In [87]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(our_embeddings, data.X, data.y)))

Missing 883 words. Will replace them with mean vector
/opt/conda/lib/python3.7/site-packages/web-0.0.1-py3.7.egg/web/evaluate.py:336: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  A = np.vstack(w.get(word, mean_vector) for word in X[:, 0])
/opt/conda/lib/python3.7/site-packages/web-0.0.1-py3.7.egg/web/evaluate.py:337: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  B = np.vstack(w.get(word, mean_vector) for word in X[:, 1])
Missing 74 words. Will replace them with mean vector
Missing 122 words. Will replace them with mean vector


Spearman correlation of scores on MEN 0.0226286156446734
Spearman correlation of scores on WS353 0.11148925385144111
Spearman correlation of scores on SIMLEX999 0.011862104575537205


In [88]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(speech2vec_embeddings, data.X, data.y)))

Missing 392 words. Will replace them with mean vector
Missing 61 words. Will replace them with mean vector
Missing 24 words. Will replace them with mean vector


Spearman correlation of scores on MEN 0.6056592803599269
Spearman correlation of scores on WS353 0.43349390636024643
Spearman correlation of scores on SIMLEX999 0.25938770901422736
